<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook demonstrates the advanced concepts of fairness detection using IBM Watson OpenScale's indirect bias mechanism.

We make use of the HR hiring dataset, where protected attributes `gender` and `ethnicity` are not used to train the machine learning model that we are building as part of this notebook. However, we will configure OpenScale to monitor fairness for these attributes to check if the model is indirectly behaving in a biased manner.

The model will be configured as a pre-production model in OpenScale.

This notebook should be run in a [Watson Studio project](https://dataplatform.cloud.ibm.com/projects/), using a Python 3.7 or above runtime environment. If you are viewing this in Watson Studio and do not see Python 3.7 or above in the upper right corner of your screen, please update the runtime now. It requires the following Cloud services:

 - [IBM Watson OpenScale](https://cloud.ibm.com/catalog/services/watson-openscale)
 - [Watson Machine Learning](https://cloud.ibm.com/catalog/services/machine-learning)

If you have a paid Cloud account, you may also provision a [Databases for PostgreSQL](https://cloud.ibm.com/catalog/services/databases-for-postgresql) or [Db2 Warehouse](https://cloud.ibm.com/catalog/services/db2-warehouse) service to take full advantage of integration with Watson Studio and continuous learning services. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

## Credentials for IBM Cloud Services

Your Cloud API key can be generated by going to the [Users section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it between the single quotes in the cell below.

In [ ]:
CLOUD_API_KEY = "__CLOUD_API_KEY_HERE___"

## Create a deployment space

All deployed models require a deployment space. Go to the [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create a new space, or choose an existing one.

**It is HIGHLY recommended that you create a new space for this project and model, because this script will remove existing spaces as machine learning service providers for OpenScale, which may interfere with other models you are monitoring if they are in the same space.**

Click on the name of the space, then go to the **Settings tab**. Locate the **Space ID** and then click the icon to copy the ID to your clipboard. Paste your space ID between the quotation marks below.

In [ ]:
WML_SPACE_ID = '__DEPLOYMENT_SPACE_ID_HERE___'

## Database Credentials

This tutorial can use Databases for PostgreSQL, Db2 Warehouse, or a free internal version of PostgreSQL to create a datamart for OpenScale. The free internal version can be accessed via the OpenScale APIs, but you will be unable to access it using direct database queries.

If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you do not have a paid Cloud account or would prefer not to provision this paid service, you may use the free internal PostgreSQL service with OpenScale. Do not update the cell below.

To provision a new instance of Db2 Warehouse, locate [Db2 Warehouse](https://cloud.ibm.com/catalog/services/db2-warehouse) in the Cloud catalog, give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Db2 Warehouse credentials into the cell below.

To provision a new instance of Databases for PostgreSQL, locate [Databases for PostgreSQL](https://cloud.ibm.com/catalog/services/databases-for-postgresql) in the Cloud catalog, give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Databases for PostgreSQL credentials into the cell below.

In [ ]:
DB_CREDENTIALS = None

## Run the notebook

At this point, you should be able to run the remainder of the notebook without modifications. It will download training data from github and train a model. The model will be saved to your Watson Machine Learning deployment space and deployed. Finally, OpenScale will be configured to monitor the model.

## Install packages and import libraries

In [ ]:
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install pyspark==2.4.0 --no-cache | tail -n 1

In [ ]:
import pandas as pd
from pyspark import SparkContext, SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.sql.types import StructType, DoubleType, StringType, ArrayType

In [ ]:
WML_CREDENTIALS = {
    "apikey": CLOUD_API_KEY,
    "url": 'https://us-south.ml.cloud.ibm.com'
}
WML_CREDENTIALS

## Load and explore data

In [ ]:
!rm hr_hiring_data.csv
!wget https://raw.githubusercontent.com/ericmartens/indirect-bias/main/data/hr_training_data.csv

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

pd_data = pd.read_csv("hr_training_data.csv", sep=",", header=0)

spark = SparkSession.builder.getOrCreate()
spark_df = spark.createDataFrame(pd_data)
spark_df.head()

## Create the model pipeline and train the model

In [ ]:
MODEL_NAME = 'Hiring - Challenger'
DEPLOYMENT_NAME = 'Hiring Deployment - Challenger'

Index the categorical fields from the training data

In [ ]:
si_BusinessTravel = StringIndexer(inputCol='BusinessTravel', outputCol='BusinessTravel_IX')
si_Department = StringIndexer(inputCol='Department', outputCol='Department_IX')
si_Education = StringIndexer(inputCol='Education', outputCol='Education_IX')
si_EducationField = StringIndexer(inputCol='EducationField', outputCol='EducationField_IX')
si_RelevantEducationLevel = StringIndexer(inputCol='RelevantEducationLevel', outputCol='RelevantEducationLevel_IX')
si_JobRole = StringIndexer(inputCol='JobRole', outputCol='JobRole_IX')
si_JobLevel = StringIndexer(inputCol='JobLevel', outputCol='JobLevel_IX')
si_MaritalStatus = StringIndexer(inputCol='MaritalStatus', outputCol='MaritalStatus_IX')
si_OverTime = StringIndexer(inputCol='OverTime', outputCol='OverTime_IX')
si_RequestedBenefits = StringIndexer(inputCol='RequestedBenefits', outputCol='RequestedBenefits_IX')
si_PreferredSkills = StringIndexer(inputCol='PreferredSkills', outputCol='PreferredSkills_IX')
si_JobType = StringIndexer(inputCol='JobType', outputCol='JobType_IX')
si_SalaryExpectation = StringIndexer(inputCol='SalaryExpectation', outputCol='SalaryExpectation_IX')

si_InterviewScore = StringIndexer(inputCol='InterviewScore', outputCol='InterviewScore_IX')
si_ResumeScore = StringIndexer(inputCol='ResumeScore', outputCol='ResumeScore_IX')

si_Label = StringIndexer(inputCol="HIRED", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

Drop the gender and ethnicity columns, since we will not be using these to train the model. OpenScale will be configured to monitor these values.

In [ ]:
columns_to_drop = ['Gender', 'Ethnicity']
spark_df_tmp = spark_df.drop(*columns_to_drop)

Perform the train/test split.

In [ ]:
(train_data, test_data) = spark_df_tmp.randomSplit([0.9, 0.1], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

In [ ]:
va_features = VectorAssembler(
inputCols=["Age", "BusinessTravel_IX", "Department_IX", "DistanceFromHome",
           "Education_IX", "EducationField_IX", "RelevantEducationLevel_IX", "JobLevel_IX", "JobRole_IX"
           , "MaritalStatus_IX","NumCompaniesWorked", "OverTime_IX",
           "InterviewScore_IX", "ResumeScore_IX", "RequestedBenefits_IX", "TotalWorkingYears", "PreferredSkills_IX",
           "YearsAtCurrentCompany","RelevantExperience","JobType_IX","SalaryExpectation_IX"], outputCol="features")

In [ ]:
classifier = GBTClassifier(featuresCol="features")

Construct the model pipeline

In [ ]:
pipeline = Pipeline(stages=[si_BusinessTravel, si_Department,si_Education, si_EducationField,si_RelevantEducationLevel, si_JobRole,si_JobLevel, si_MaritalStatus,
        si_OverTime,si_InterviewScore,si_ResumeScore,si_RequestedBenefits,si_PreferredSkills,si_JobType,si_SalaryExpectation, si_Label, va_features, classifier, label_converter])

Evaluate the model

In [ ]:
model = pipeline.fit(train_data)
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)

# Save and deploy the model

Before we save and deploy the model, we will remove any existing OpenScale subscriptions for this model, as well as any existing deployments, so that the notebook can be run repeatedly to refresh the demo.

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import APIClient

service_credentials = {
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

authenticator = IAMAuthenticator(apikey=service_credentials['apikey'])

wos_client = APIClient(authenticator=authenticator)
wos_client.version

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    if subscription.entity.asset.name == MODEL_NAME:
        print("Deleting existing subscription for model", subscription.entity.asset.name)
        wos_client.subscriptions.delete(subscription.metadata.id)

We can now save and deploy the model.

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
wml_client.spaces.list(limit=10)

In [ ]:
wml_client.set.default_space(WML_SPACE_ID)

In [ ]:
training_data_references = [
                {
                    "id": "product line",
                    "type": "s3",
                    "connection": {
                        "access_key_id": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                        "endpoint_url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::"
                    },
                    "location": {
                        "bucket": "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                        "path": "hr_training_data.csv",
                    }
                }
            ]

In [ ]:
sw_spec_uid = wml_client.software_specifications.get_uid_by_name("spark-mllib_2.4")
sw_spec_uid

Delete any existing versions of this model and deployment

In [ ]:
for deployment in wml_client.deployments.get_details()['resources']:
    deployment_id=deployment['metadata']['id']
    deployment = wml_client.deployments.get_details(deployment_id)
    model_uid = deployment['entity']['asset']['id']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting existing deployment with id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting existing model with id', model_uid)
        wml_client.repository.delete(model_uid) 

Store the model

In [ ]:
print("Storing model...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props={
        wml_client._models.ConfigurationMetaNames.NAME: "{}".format(MODEL_NAME),
        wml_client._models.ConfigurationMetaNames.SPACE_UID: WML_SPACE_ID,
        wml_client._models.ConfigurationMetaNames.TYPE: "mllib_2.4",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
        wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
      wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "HIRED",
    }, 
    training_data=train_data, 
    pipeline=pipeline)
model_uid = wml_client.repository.get_model_uid(published_model_details)
print("Done")

Deploy the model

In [ ]:
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_uid(deployment_details)

print("Scoring URL: {}".format(scoring_url))
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

# Configure OpenScale 

We will now configure Watson OpenScale to monitor the deployed model. When this step is finished, all data into and out of the model will be logged, and can be made available to our applications via the Python API. Additionally, we will have the ability to generate explanations for individual predictions.

The code below creates the OpenScale datamart, a database in which OpenScale will store its data. If you have already set up OpenScale, it will use your existing datamart and not remove any previous data. If you specified Db2 Warehouse or Databases for PostgreSQL credentials above, it will use those credentials to create a datamart with that paid service. Finally, if you have not previously used OpenScale and did not supply credentials for a paid database service, it will create the datamart in a free, internal database. This internal database still allows access via the OpenScale APIs, but you cannot access it directly via database queries.

## Create schema and datamart

### Set up datamart
Watson OpenScale uses a database to store payload logs and calculated metrics. If a datamart already exists, OpenScale will use that. If not, and database credentials were not supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created on that service.

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print("Setting up external datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["hostname"],
                        username=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["username"],
                        password=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["password"],
                        db=DB_CREDENTIALS["connection"]["postgres"]["database"],
                        port=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["port"],
                        ssl=True,
                        sslmode=DB_CREDENTIALS["connection"]["postgres"]["query_options"]["sslmode"],
                        certificate_base64=DB_CREDENTIALS["connection"]["postgres"]["certificate"]["certificate_base64"]
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print("Setting up internal datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print("Using existing datamart {}".format(data_mart_id))

## Bind WML machine learning instance as Pre-Prod

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If a binding with name "Watson Machine Learning OpenScale Demo" already exists, this code will delete that binding a create a new one.

In [ ]:
wos_client.service_providers.show()

In [ ]:
SERVICE_PROVIDER_NAME = "WMLearning OpenScale Demo - Preprod"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [ ]:
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "pre_production",
        credentials=WMLCredentialsCloud(
            apikey=CLOUD_API_KEY,      
            url=WML_CREDENTIALS["url"],
            instance_id=None
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_id=deployment_uid, deployment_space_id = WML_SPACE_ID).result['resources'][0]
asset_deployment_details

## Set up the model for monitoring in OpenScale

In [ ]:
feature_columns=list(pd_data.drop(['HIRED','Gender','Ethnicity'],axis=1))
feature_columns

In [ ]:
categorical_features = pd_data[feature_columns].select_dtypes(include=['object']).columns.tolist()
categorical_features

In [ ]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,deployment_space_id=WML_SPACE_ID)
model_asset_details_from_deployment

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details["metadata"]["guid"],
            name=asset_deployment_details["entity"]["name"],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details["metadata"]["url"]
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="HIRED",
            probability_fields=["probability"],
            prediction_field="predictedLabel",
            feature_fields = feature_columns,
            categorical_fields = categorical_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket="faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                                                                                                    file_name="hr_training_data.csv"),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::",
                                                                        "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                                                                        "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                                                                        "iam_url": "https://iam.bluemix.net/oidc/token"})),
            training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
print(subscription_details)

## Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few record for predictions.

## Get payload data for scoring the model

Download the payload data from github, and split out the protected attributes and the prediction column.

In [ ]:
!rm payload_100.csv
!wget https://raw.githubusercontent.com/ericmartens/indirect-bias/main/data/payload_100.csv

In [ ]:
payload_data = pd.read_csv("payload_100.csv", sep=",", header=0)
protected_attributes=['Ethnicity','Gender']

cols_to_remove = ['HIRED']
cols_to_remove.extend(protected_attributes)

## Create the meta data frame capturing the sensitive data

In [ ]:
meta_df = payload_data[protected_attributes].copy()
meta_fields = meta_df.columns.tolist()
meta_values = meta_df[meta_fields].values.tolist()

## Construct the scoring payload

The scoring payload includes the protected data (gender and ethnicity) in the metadata, so it can be logged by OpenScale, but not passed to the model for scoring.

In [ ]:
def get_scoring_payload(no_of_records_to_score = 1):
    meta_payload = {
        "fields": meta_fields,
        "values": meta_values[:no_of_records_to_score]
    }

    for col in cols_to_remove:
        if col in payload_data.columns:
            del payload_data[col] 

    fields = payload_data.columns.tolist()
    values = payload_data[fields].values.tolist()

    payload_scoring = {"input_data": [{"fields": fields, "values": values[:no_of_records_to_score],"meta": meta_payload}]}
 
    return payload_scoring

## Method to perform scoring

In [ ]:
def sample_scoring(no_of_records_to_score = 1):
    records_list=[]
    payload_scoring = get_scoring_payload(no_of_records_to_score)
    
    scoring_response = wml_client.deployments.score(deployment_uid, payload_scoring)
    print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])
    print(json.dumps(scoring_response, indent=None))

## Score the model and print the scoring response

In [ ]:
import json
sample_scoring(no_of_records_to_score=1)

Validate that the payload scoring worked by checking the payload data set ID

In [ ]:
import time

time.sleep(10)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

## Fairness, drift monitoring and explanations

###  Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, sex and age. In each case, we must specify:

 - Which model feature to monitor One or more majority groups
 - Which are values of that feature that we expect to receive a higher percentage of favorable outcomes
 - One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
 - The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%)
 
Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added.

### Create Fairness Monitor Instance

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "features": [
        {
                "feature": "Gender",
                "majority": ["Male"],
                "minority": ["Female"]
            },
        {
            "feature": "Ethnicity",
            "majority": ["non-minority"],
            "minority": ["minority"]
        }
    ],
    "favourable_class": ["YES"],
    "unfavourable_class": ["NO"],
    "min_records": 100
}
thresholds = [
    {
        "metric_id": "fairness_value",
        "specific_values": [
           {
                "applies_to": [
                    {
                        "type": "tag",
                        "value": "Gender",
                        "key": "feature"
                    }
                ],
                "value": 80
            },
            {
                "applies_to": [
                    {
                        "type": "tag",
                        "value": "Ethnicity",
                        "key": "feature"
                    }
                ],
                "value": 80
            }
        ],
        "type": "lower_limit",
        "value": 80
    }
]
fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id

### Drift configuration

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

## In regards to indirect bias based on following results of fairness run you can see there is fairness issue with both the protected features. If you dig deeper in UI, it shows you the following:
- `Ethnicity` have strong correlation with `interviewscore` and the model fainess issue against `minority`. It might mean, `minority` people tends to get low score compare to `non-minority` group.
- `Gender` have strong correlation with `salaryexpectation` and the model fainess issue against `Female`, It might mean, `Female` tends to get low salary compare to `male`

# Quality monitoring and feedback logging

## Enable quality monitoring
The code below turns on the quality (accuracy) monitor and sets an alert threshold of 80%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [ ]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

## Configure Explainability
Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id

## Upload and evaluate test data

At this point, you can navigate to the [OpenScale Insights Dashboard](https://aiopenscale.cloud.ibm.com/aiopenscale/) and select the **Hiring Deployment - Challenger** model. From the **Actions** menu, choose **Evaluate now**. Select **from CSV** from the **Import** dropdown, and upload the [payload_100.csv](https://raw.githubusercontent.com/ericmartens/indirect-bias/main/data/payload_100.csv) file. Then click **Upload and evaluate**. The monitors will take a few minutes to run, but when the screen refreshes, you will see information on the test results for fairness, quality and drift, along with two generated explanations.